# Text classification

Text classification is a common NLP task that assigns a label or class to text. Some of the largest companies run text classification in production for a wide range of practical applications. One of the most popular forms of text classification is sentiment analysis, which assigns a label like 🙂 positive, 🙁 negative, or 😐 neutral to a sequence of text.

This guide will show you how to:

1. Finetune [DistilBERT](https://huggingface.co/distilbert/distilbert-base-uncased) on the [IMDb](https://huggingface.co/datasets/imdb) dataset to determine whether a movie review is positive or negative.
2. Use your finetuned model for inference.

<Tip>

To see all architectures and checkpoints compatible with this task, we recommend checking the [task-page](https://huggingface.co/tasks/text-classification).

</Tip>

Before you begin, make sure you have all the necessary libraries installed:

```bash
pip install transformers datasets evaluate accelerate
```

We encourage you to login to your Hugging Face account so you can upload and share your model with the community. When prompted, enter your token to login:

Additional note: be sure to have the right pytorch version for the architecture you're using, and the right CUDA version if you're using a GPU.
With this code I had problems with NumPy >= 2 so: pip install numpy<2.0 was needed.

In [1]:
# from huggingface_hub import notebook_login
#
# notebook_login()
import os
from dotenv import load_dotenv
from transformers.models.gemma2.modular_gemma2 import Gemma2Model

envs_path = os.getcwd() + '/../envs.env'
load_dotenv(envs_path)
hf_token = os.getenv("HF_TOKEN")

## Load IMDb dataset

Start by loading the IMDb dataset from the 🤗 Datasets library:

In [2]:
from datasets import load_dataset

imdb = load_dataset("imdb", token=hf_token)

Then take a look at the dataset

In [3]:
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [4]:
imdb["test"][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

There are two fields in this dataset:

- `text`: the movie review text.
- `label`: a value that is either `0` for a negative review or `1` for a positive review.

## Preprocess

The next step is to load a DistilBERT tokenizer to preprocess the `text` field:

In [5]:
from transformers import AutoTokenizer, EarlyStoppingCallback

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

Create a preprocessing function to tokenize `text` and truncate sequences to be no longer than DistilBERT's maximum input length:

In [6]:
def preprocess_function(examples):
    # examples is a batch of examples
    return tokenizer(examples["text"], truncation=True)

To apply the preprocessing function over the entire dataset, use 🤗 Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) function. You can speed up `map` by setting `batched=True` to process multiple elements of the dataset at once:

In [7]:
tokenized_imdb = imdb.map(preprocess_function, batched=True)

Check the tokenized output size and structure

In [8]:
tokenized_imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [9]:
tokenized_imdb["train"][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

Now create a batch of examples using [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorWithPadding). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

A Data Collator is a function that takes a list of samples from a dataset and convert them into a batch by applying the necessary padding. This is especially useful when dealing with sequences of varying lengths, as it ensures that all sequences in a batch are of the same length, which is required for efficient processing by the model.

In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Evaluate

Including a metric during training is often helpful for evaluating your model's performance. You can quickly load a evaluation method with the 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load the [accuracy](https://huggingface.co/spaces/evaluate-metric/accuracy) metric (see the 🤗 Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric):

In [11]:
import evaluate

accuracy = evaluate.load("accuracy")

Then create a function that passes your predictions and labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the accuracy:

In [12]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    print(predictions.shape)
    print(labels.shape)
    print(predictions)
    print(labels)
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

Your `compute_metrics` function is ready to go now, and you'll return to it when you setup your training.

## Train

Before you start training your model, create a map of the expected ids to their labels with `id2label` and `label2id`:

In [13]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

<Tip>

If you aren't familiar with finetuning a model with the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer), take a look at the basic tutorial [here](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-with-pytorch-trainer)!

</Tip>

You're ready to start training your model now! Load DistilBERT with [AutoModelForSequenceClassification](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForSequenceClassification) along with the number of expected labels, and the label mappings:

In [14]:
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Setup WANDB for experiment tracking

In [15]:
import wandb

base_output_dir = "models"
run_name = "imdb_cls_distilbert"
output_dir = os.path.join(base_output_dir, run_name)

wandb_api_key = os.getenv("WANDB_API_KEY")

os.environ["WANDB_PROJECT"] = "ai509"
wandb.init(name=run_name)

wandb: Currently logged in as: gbar (gbar-university-of-southern-denmark) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Check and ensure which device you're using for training

In [ ]:
import torch

# Check if MPS is available (for Mac with M1/M2/M3 chips)
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS device")
# Check if CUDA is available (for NVIDIA GPUs)
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using CUDA device: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU (no GPU acceleration available)")

model.to(device)

At this point, only three steps remain:

1. Define your training hyperparameters in [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments). The only required parameter is `output_dir` which specifies where to save your model. You'll push this model to the Hub by setting `push_to_hub=True` (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) will evaluate the accuracy and save the training checkpoint.
2. Pass the training arguments to [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.
3. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

In [16]:
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # gradient_accumulation_steps=2,
    num_train_epochs=2, # usually 2-5 epochs are sufficient for finetuning
    weight_decay=0.01, # penalize large weights, regularization, helps generalization
    eval_strategy="epoch", # other options: "no", "steps"
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="wandb",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=2)], # need to switch from epochs to steps and set eval_steps for this to work
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.221400,0.196090,0.925840
2,0.142000,0.235490,0.931080


(25000, 2)
(25000,)
[[ 2.736453   -2.2031093 ]
 [ 1.3061811  -1.1360815 ]
 [ 2.0702782  -1.6990875 ]
 ...
 [ 0.1373245  -0.18632975]
 [-0.87732816  0.6917744 ]
 [-1.3990978   1.2089478 ]]
[0 0 0 ... 1 1 1]
(25000, 2)
(25000,)
[[ 3.5519526  -2.7534695 ]
 [ 2.45051    -1.9594156 ]
 [ 2.8799472  -2.2565958 ]
 ...
 [ 0.6259913  -0.47749105]
 [-2.7624931   2.5328398 ]
 [-2.5502985   2.3243651 ]]
[0 0 0 ... 1 1 1]


TrainOutput(global_step=3126, training_loss=0.20088823285533006, metrics={'train_runtime': 3158.8026, 'train_samples_per_second': 15.829, 'train_steps_per_second': 0.99, 'total_flos': 6556904415524352.0, 'train_loss': 0.20088823285533006, 'epoch': 2.0})

<Tip>

[Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) applies dynamic padding by default when you pass `tokenizer` to it. In this case, you don't need to specify a data collator explicitly.

</Tip>

Once training is completed, share your model to the Hub with the [push_to_hub()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.push_to_hub) method so everyone can use your model:

In [32]:
trainer.evaluate(metric_key_prefix="test")

(25000, 2)
(25000,)
[[ 2.736453   -2.2031093 ]
 [ 1.3061811  -1.1360815 ]
 [ 2.0702782  -1.6990875 ]
 ...
 [ 0.1373245  -0.18632975]
 [-0.87732816  0.6917744 ]
 [-1.3990978   1.2089478 ]]
[0 0 0 ... 1 1 1]


{'test_loss': 0.19608967006206512,
 'test_accuracy': 0.92584,
 'test_runtime': 376.2226,
 'test_samples_per_second': 66.45,
 'test_steps_per_second': 4.154,
 'epoch': 2.0}

In [17]:
# trainer.push_to_hub()

<Tip>

For a more in-depth example of how to finetune a model for text classification, take a look at the corresponding
[PyTorch notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/text_classification.ipynb).

</Tip>

## Inference

Great, now that you've finetuned a model, you can use it for inference!

Grab some text you'd like to run inference on:

In [21]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."

text_2 = "This was shit. Completely not faithful to the books, and really annoying. I turned the screen off immediately."

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for sentiment analysis with your model, and pass your text to it:

In [25]:
from transformers import pipeline

output_dir = "imdb_cls_distilbert/checkpoint-3126"

classifier = pipeline("sentiment-analysis", model=output_dir)
classifier(text)

Device set to use mps:0


[{'label': 'POSITIVE', 'score': 0.9977442026138306}]

You can also manually replicate the results of the `pipeline` if you'd like:

Tokenize the text and return PyTorch tensors:

In [26]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(output_dir)
inputs = tokenizer(text, return_tensors="pt")

In [33]:
inputs

{'input_ids': tensor([[  101,  2023,  2001,  1037, 17743,  1012,  2025,  3294, 11633,  2000,
          1996,  2808,  1010,  2021,  4372,  2705,  7941,  2989,  2013,  2927,
          2000,  2203,  1012,  2453,  2022,  2026,  5440,  1997,  1996,  2093,
          1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]])}

Pass your inputs to the model and return the `logits`:

In [27]:
import torch
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(output_dir)
with torch.no_grad():
    logits = model(**inputs).logits

Get the class with the highest probability, and use the model's `id2label` mapping to convert it to a text label:

In [29]:
logits

tensor([[-3.1252,  2.9668]])

In [30]:
# apply softmax to convert logits to probabilities
probabilities = torch.nn.functional.softmax(logits, dim=1)
probabilities

tensor([[0.0023, 0.9977]])

In [28]:
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'POSITIVE'